# Capstone - Partial Discharge
## Julian Sweet DSI-LA-6
## Notebook 4 -Logisitic Regression Modeling, Balanced Class

In the preceding notebooks the Kaggle PD training data was manipulated to make a pair of data sets with both balanced and unbalanced classes.

The original data was then visualized as both time-domain as well as frequency-domain data.

Before proceeding, it is important to emphasize what can and cannot be modelled given finite storage and computer memory contraints.

To model the time-domain data, even using the smaller balanced classes subset, is not possible at this time. This smaller balanced class subset is composed of 840 rows (observations) of labelled training. However, the columns of each observation represent 800,000 feature by which a model would be expected to compute over.

Attempts were made using a Macbook Pro with 8 GB of RAM, a desktop home workstation with 32GB of RAM, as well as a high-powered GPU instance from AWS with 61GB of RAM. Even with the smallest data set of 840 rows by 800k columns. Kernal overflow was inevitable. Obviously the same problem exists with the STFT data it too is composed of over 800k of columns per observation.

Downsampling of time-domain or STFT data should be explored. However, moving forward all modeling is presented below is for FFT data whereby each row (observation) is only 256 columns in length, which is 2^8 and less than resolution utilized during intial EDA, but still a considerable feature set.

Modeling will begin with the smallest of the two, the balanced class dataset of 1050 rows (80 / 20 train test split) by 1024 columns.


In [2]:
import numpy as np
import pandas as pd

from scipy.signal import resample, stft
from sys import getsizeof
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_auc_score
from scipy.fftpack import fft

In [3]:
%%time
#X_train_resamp_data = np.load('./npy_datasets/X_train_resamp_data.npy')
#X_test_resamp_data  = np.load('./npy_datasets/X_test_resamp_data.npy')
y_test_resamp       = np.load('./npy_datasets/y_test_resamp.npy')
y_train_resamp      = np.load('./npy_datasets/y_train_resamp.npy')

CPU times: user 2.95 ms, sys: 2.73 ms, total: 5.68 ms
Wall time: 4.56 ms


Quick to load, but untenable by any model I tried.

In [6]:
X_train_resamp_data.shape, X_test_resamp_data.shape, y_test_resamp.shape, y_train_resamp.shape

((840, 800000), (210, 800000), (210,), (840,))

In [39]:
%%time
n_fft = 256
X_train_fft = np.abs(fft(X_train_resamp_data, n_fft))
X_test_fft  = np.abs(fft(X_test_resamp_data, n_fft))

/Users/juliansweet/anaconda3/lib/python3.6/site-packages/scipy/fftpack/basic.py:153: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  x = x[index]


CPU times: user 2.26 s, sys: 5.32 s, total: 7.58 s
Wall time: 9.8 s


In [40]:
X_train_fft.shape, X_test_fft.shape

((840, 256), (210, 256))

In [41]:
# np.save('./npy_datasets/X_train_bal_fft', X_train_fft) 

In [42]:
# np.save('./npy_datasets/X_test_bal_fft', X_test_fft)

In [4]:
X_train_fft = np.load('./npy_datasets/X_train_bal_fft.npy')
X_test_fft  = np.load('./npy_datasets/X_test_bal_fft.npy')

Modeling will begin with Linear / Logistic Regression / Classification. It is the simplest, the most human interpretable machine model. Regression should always be attempted unless dataset specfics dictate that another model is optimal.

The code below specifies try try both types of regularization and at least initially, will score with accuracy.

6 processor jobs refers to 4 cores being multithreaded constituting 8 virtual cores total. However, using all 8 will make the computer unusable during processing. Holding back two virual cores (one real) ensures computer usability. 

This model will be cross validated 5 times. Verbose level of any number greater than 1 maximizes verbosity.

In [5]:
params1 = {
    'penalty' : ['l1', 'l2']
    }

logr1 = GridSearchCV(LogisticRegression(max_iter = 35000), 
                           n_jobs = 6, 
                           verbose = 2,
                           scoring = 'accuracy',
                           param_grid = params1,
                           cv = 5)

In [6]:
logr1.fit(X_train_fft, y_train_resamp)
logr1.best_params_

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   5 out of  10 | elapsed:    1.6s remaining:    1.6s
[Parallel(n_jobs=6)]: Done  10 out of  10 | elapsed:    2.0s finished
/Users/juliansweet/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


{'penalty': 'l2'}

In [7]:
logr1.score(np.abs(X_train_fft), y_train_resamp)

0.8083333333333333

In [8]:
logr1.score(np.abs(X_test_fft), y_test_resamp)

0.7619047619047619

Not overfit, also able to give a signifcantly better answer on than the naive baseline for balanced classes of 50% accuracy.

In [9]:
params2 = {
    'penalty' : ['l1', 'l2']
    }

logr2 = GridSearchCV(LogisticRegression(max_iter = 35000), 
                           n_jobs = 6, 
                           verbose = 2,
                           scoring = 'roc_auc',
                           param_grid = params2,
                           cv = 5)

In [10]:
logr2.fit(np.abs(X_train_fft), y_train_resamp)
logr2.best_params_

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   5 out of  10 | elapsed:    0.3s remaining:    0.3s
[Parallel(n_jobs=6)]: Done  10 out of  10 | elapsed:    0.7s finished
/Users/juliansweet/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


{'penalty': 'l1'}

In [11]:
logr2.score(X_train_fft, y_train_resamp)

0.8930668934240362

In [12]:
logr2.score(X_test_fft, y_test_resamp)

0.8384580498866213

The model is still overfit, but much better than the naive baseline.

Changing the scoring mechanism during the fit also means that the .score method now returns "roc_auc" and not "accuracy

In [13]:
roc_auc_score(y_train_resamp, logr2.predict(X_train_fft))

0.8035714285714286

Now for a confusion matrix. SKLearn exchanges the labelling convention, so this will create a dataframe of what the model guessed and what was the correct answer with explicit labels.

In [14]:
d = {'predictions': logr2.predict(X_test_fft), 'actual': y_test_resamp}

In [15]:
con = df = pd.DataFrame(data = d)
con.head(10)

,predictions,actual
0,1,1
1,0,1
2,1,1
3,1,1
4,1,1
5,1,1
6,0,1
7,1,1
8,1,1
9,0,1


In [16]:
C = confusion_matrix(con['actual'], con['predictions'])

Below is the confusion matrix, true positive, true negative down the diagonal. False positive top right, false negative bottom left.

In [17]:
C

array([[83, 22],
       [28, 77]])

In [23]:
tn, fp, fn, tp = confusion_matrix(con['actual'], con['predictions']).ravel()

In [24]:
(tn, fp, fn, tp)

(83, 22, 28, 77)

The confusion matrix normalized.

In [25]:
C / C.astype(np.float).sum(axis=1)

array([[0.79047619, 0.20952381],
       [0.26666667, 0.73333333]])

Sensitivity = .733 / (.733 + .266) = .73

In [26]:
df = pd.DataFrame(logr2.predict_proba(X_test_fft))
df.head()

,0,1
0,4.217052e-02,0.957829
1,5.747590e-01,0.425241
2,8.804192e-09,1.000000
3,2.907145e-01,0.709286
4,6.930036e-02,0.930700


In [27]:
biased_guess = df[1] >= .40

In [34]:
tn, fp, fn, tp = confusion_matrix(y_test_resamp, biased_guess).ravel()

In [35]:
(tn, fp, fn, tp)

(77, 28, 21, 84)

In [ ]:
Sensitivity = .8 / (.8 + .2) = 0.8

This skewing of the AUC ROC curve decreases True Negative and increases False Positive, but has benefit of greatly decreasing False Negative and increasing True Positive. Sensitivity is increased.

Notebook #5 will now perform logistic regression analysis, but with the larger anomalous / unbalanced dataset.